In [6]:
#pip install pandas numpy matplotlib scipy ipywidgets fileupload


In [16]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve
from ipywidgets import interact, widgets

# Seu caminho para a pasta que contém os arquivos CSV
pasta_csv = 'Sua_Pasta'

# Listar todos os arquivos CSV na pasta
arquivos_csv = [arquivo for arquivo in os.listdir(pasta_csv) if arquivo.endswith('.csv')]

# Função para suavizar a derivada e exportar dados
def suavizar_derivada_exportar(nome_arquivo, smooth, interpolação, nome_arquivo_exportacao_widget):
    # Construir o caminho completo para o arquivo CSV selecionado
    caminho_arquivo = os.path.join(pasta_csv, nome_arquivo)

    # Carregar o arquivo CSV
    df = pd.read_csv(caminho_arquivo, header=None, names=['x', 'y'], delimiter=';')

    # Extraindo os dados das colunas
    x = df['x'].values.astype(float)
    y = df['y'].values.astype(float)

    # Calcular a derivada
    derivada = np.gradient(y, x)

    # Suaviazação Adjacent Averaging
    derivada_suavizada = np.convolve(derivada, np.ones(smooth)/smooth, mode='same')

    # Interpolar os dados suavizados usando polinômio
    coef_interp = np.polyfit(x, derivada_suavizada, interpolação)
    derivada_interp = np.polyval(coef_interp, x)

    # Plotar o gráfico da derivada suavizada e interpolada
    plt.plot(x, derivada_suavizada, label='Derivada Suavizada')
    plt.plot(x, derivada_interp, label='Derivada Interpolada', linestyle='solid')

    plt.xlabel('Bias (V)')
    plt.ylabel('dI/dV')
    plt.title('Gráfico da Derivada Suavizada e Interpolada')
    plt.legend()
    plt.show()

    # Adicionar .txt ao nome do arquivo se não fornecido
    nome_arquivo_exportacao = nome_arquivo_exportacao_widget if nome_arquivo_exportacao_widget.endswith('.txt') else nome_arquivo_exportacao_widget + '.txt'
    
    # Exibir botão para salvar os dados
    btn_salvar = widgets.Button(description="Salvar Dados")
    
    # Função para salvar os dados quando o botão é clicado
    def salvar_dados(_):
        min_length = min(len(x), len(y), len(derivada), len(derivada_suavizada), len(derivada_interp))
        dados_exportacao = np.column_stack((x[:min_length], y[:min_length], derivada[:min_length], derivada_suavizada[:min_length], derivada_interp[:min_length]))
        np.savetxt(nome_arquivo_exportacao, dados_exportacao, delimiter='\t', header='x\ty\tderivada\tderivada_suavizada\tderivada_interp')
        print(f'Dados exportados com sucesso para: {nome_arquivo_exportacao}')
    
    # Associar a função ao botão
    btn_salvar.on_click(salvar_dados)
    
    # Exibir botão
    display(btn_salvar)

# Adicionar sliders interativos e campo de entrada de texto
interact(suavizar_derivada_exportar, 
         nome_arquivo=widgets.Dropdown(options=arquivos_csv, description='Arquivo CSV'),
         smooth=widgets.IntSlider(min=1, max=50, step=1, value=10),
         interpolação=widgets.IntSlider(min=1, max=35, step=1, value=10),
         nome_arquivo_exportacao_widget=widgets.Text(value='', description='Exportar'))


interactive(children=(Dropdown(description='Arquivo CSV', options=('Thiago_COPRINUS_Au 02259_F.csv', 'Thiago_C…

<function __main__.suavizar_derivada_exportar(nome_arquivo, smooth, interpolação, nome_arquivo_exportacao_widget)>